<a href="https://colab.research.google.com/github/YawBoah/MSCAssignments/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install dash

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from dash import Dash, dcc, html
import plotly.graph_objs as go

# Load datasets
workforce = pd.read_csv('/content/Workforce_Data.csv')
forecast = pd.read_csv('/content/urban_grocers_forecast.csv')

# Convert date columns
forecast['ds'] = pd.to_datetime(forecast['ds'])
workforce['Date'] = pd.to_datetime(workforce['Date'])

# Clean attendance
def attendance_to_numeric(value):
    return 1 if value.strip().lower() == 'present' else 0

workforce['attendance_numeric'] = workforce['Staff_Attendance'].apply(attendance_to_numeric)

# Aggregate attendance data
daily_attendance = workforce.groupby('Date').agg({
    'attendance_numeric': 'sum',
    'Hours_Worked': 'sum',
    'Daily_Sales_Target': 'mean'
}).reset_index()

daily_attendance.rename(columns={'Date': 'ds', 'attendance_numeric': 'Staff_Attendance'}, inplace=True)

# Train staff prediction model
X = daily_attendance[['Daily_Sales_Target']]
y = daily_attendance['Staff_Attendance']
model = LinearRegression()
model.fit(X, y)

forecast['Daily_Sales_Target'] = forecast['yhat']
forecast['predicted_staff'] = model.predict(forecast[['Daily_Sales_Target']])
forecast['predicted_staff'] = forecast['predicted_staff'].apply(lambda x: max(1, round(x)))

# App
app = Dash(__name__)
app.title = "Urban Grocers Dashboard"

# Layout
app.layout = html.Div([
    html.H1('Urban Grocers Strategic Dashboard', style={'textAlign': 'center', 'paddingBottom': '20px'}),

    dcc.Tabs([
        # Tab 1 - Sales Forecast
        dcc.Tab(label='📈 Sales Forecast Overview', children=[
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=forecast['ds'], y=forecast['Daily_Sales_Target'], mode='lines+markers', name='Forecasted Sales'),
                        go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', line=dict(width=0), showlegend=False),
                        go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='lines', name='Confidence Interval',
                                   fillcolor='rgba(0,100,80,0.2)', line=dict(width=0))
                    ],
                    'layout': go.Layout(title='Forecasted Sales with Confidence Interval', xaxis_title='Date', yaxis_title='Sales')
                }
            )
        ]),

        # Tab 2 - Inventory Optimization
        dcc.Tab(label='📦 Inventory Optimization Insights', children=[
            dcc.Graph(
                figure={
                    'data': [
                        go.Bar(x=forecast['ds'], y=forecast['Daily_Sales_Target'], name='Predicted Sales'),
                        go.Bar(x=forecast['ds'], y=np.random.randint(100, 300, len(forecast)), name='Available Inventory')  # placeholder
                    ],
                    'layout': go.Layout(barmode='group', title='Predicted Sales vs Inventory Levels', xaxis_title='Date', yaxis_title='Units')
                }
            )
        ]),

        # Tab 3 - Workforce Planning
        dcc.Tab(label='👷 Workforce Planning Dashboard', children=[
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=forecast['ds'], y=forecast['predicted_staff'], mode='lines+markers', name='Predicted Staff'),
                        go.Scatter(x=daily_attendance['ds'], y=daily_attendance['Staff_Attendance'], mode='lines+markers', name='Actual Staff Attendance')
                    ],
                    'layout': go.Layout(title='Predicted vs Actual Staff Attendance', xaxis_title='Date', yaxis_title='Number of Staff')
                }
            )
        ]),

        # Tab 4 - Event/Weather Analysis
        dcc.Tab(label='⛅ Event/Weather Impact Analysis', children=[
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=forecast['ds'], y=forecast['Daily_Sales_Target'], mode='lines+markers', name='Sales'),
                        go.Scatter(x=forecast['ds'], y=np.random.randint(20, 35, len(forecast)), mode='lines', name='Temperature (°C)', yaxis='y2')  # placeholder
                    ],
                    'layout': go.Layout(
                        title='Sales vs Weather Conditions',
                        xaxis=dict(title='Date'),
                        yaxis=dict(title='Sales'),
                        yaxis2=dict(title='Temperature (°C)', overlaying='y', side='right')
                    )
                }
            )
        ])
    ])
])

if __name__ == '__main__':
    app.run(debug=True)



<IPython.core.display.Javascript object>